In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import evaluationData

from causallib.datasets import load_nhefs
from causallib.estimation import IPW, StratifiedStandardization
from causallib.evaluation import evaluate

### BPIC2017 dataset

In [ ]:
df = pd.read_csv("bpi2017_final.csv")
df.head()

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,CreditScore,OfferedAmount,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer,timeApplication,weekdayApplication
0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,651433.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000,2.0
1,0.0,1.0,4.0,0.0,8.0,0.0,1.0,651434.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.061,2.0
2,0.0,1.0,0.0,0.0,22.0,2.0,3.0,651435.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.290,2.0
3,0.0,1.0,1.0,0.0,22.0,2.0,6.0,651437.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.613,2.0
4,0.0,1.0,0.0,0.0,21.0,2.0,3.0,651438.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.620,2.0


In [ ]:
feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

In [ ]:
t=df['treatmentOffer']
X = df[feature_names]
y=df['offerSuccess']

### Model

In [5]:
ipw = IPW(LogisticRegression(max_iter=2000), clip_min=0.05, clip_max=0.95)
ipw.fit(X, t, y)

/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in 

IPW(clip_max=0.95, clip_min=0.05, use_stabilized=False, verbose=False,
    learner=LogisticRegression(max_iter=2000))

In [6]:
outcomes = ipw.estimate_population_outcome(X,t, y)
effect = ipw.estimate_effect(outcomes[1], outcomes[0])
print(effect.values[0])

/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


0.14917069111099684


### Refutation Datasets

In [ ]:
import os
# Define folder path
folder_path = "./evaluationDatasets/Cause/"

# List to store treatment effects
ate_values = []

feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

columns_to_drop = ['offerSuccess', 'treatmentOffer']

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Read CSV file
        file_path = os.path.join(folder_path, file_name)
        placebo = pd.read_csv(file_path)
        
        # Extract features and target
        t = placebo['treatmentOffer']
        X = placebo.drop(columns=columns_to_drop)
        #placebo[feature_names}
        y = placebo['offerSuccess']
        
        # Estimate treatment effect using IPW
        ipw = IPW(LogisticRegression(max_iter=2000), clip_min=0.05, clip_max=0.95)
        ipw.fit(X, t, y)
        outcomes = ipw.estimate_population_outcome(X, t, y)
        effect = ipw.estimate_effect(outcomes[1], outcomes[0])
        
        # Append treatment effect value to list
        ate_values.append(effect.values[0])

# Calculate average treatment effect
ate = sum(ate_values) / len(ate_values)
print(ate_values)

print("Average Treatment Effect (ATE):", ate)

/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in 

[0.0015216461363667388, -0.006289414537389404]
Average Treatment Effect (ATE): -0.0023838842005113325


/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ma/ma_ma/ma_mskiera/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


### Synthetic Dataset

In [ ]:
df_synth = pd.read_csv("synthetic_dataset.csv")
print(df_synth.columns)
df_synth.head()

Index(['case:concept:name', 'NumberOfOffers', 'concept:name',
       'lifecycle:transition', 'time:timestamp', 'elementId', 'resourceId',
       'treatment', 'successful', 'treatmentSuccess', 'weekdayApplication',
       'timeApplication'],
      dtype='object')


,case:concept:name,NumberOfOffers,concept:name,lifecycle:transition,time:timestamp,elementId,resourceId,treatment,successful,treatmentSuccess,weekdayApplication,timeApplication
0,0.0,2.0,29.0,0.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
1,0.0,2.0,29.0,2.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
2,0.0,2.0,29.0,1.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
3,0.0,2.0,5.0,0.0,152.0,7.0,1.0,1.0,1.0,2.0,1.0,0.0
4,0.0,2.0,5.0,2.0,152.0,7.0,0.0,1.0,1.0,2.0,1.0,0.0


In [ ]:
synthetic_features = ['NumberOfOffers', 'concept:name',
       'lifecycle:transition', 'time:timestamp', 'elementId', 'resourceId',
       'weekdayApplication', 'timeApplication']

In [ ]:
t=df_synth['treatment']
X = df_synth[synthetic_features]
y=df_synth['treatmentSuccess']

### Add ATE result in result Dataframe

In [23]:
%store -r df_results

method = "IPW LR"
lib = "Causallib"

if method in df_results['method'].values:
     # If the method is already in the DataFrame, update the ATE and ITE columns
    df_results.loc[df_results['method'] == method, 'ATE'] = effect.values[0]
else:
    # If the method is not in the DataFrame, add a new row
    df_results = df_results._append({'method': method, 'ATE': effect.values[0], 'ITE': '', 'Library': lib}, ignore_index=True)

print(df_results)
%store df_results

                      method       ATE  \
0          Linear Regression  0.449046   
1                  Double ML  0.470982   
2                        IPW  0.311352   
3                  IPW Hajek  0.311352   
4             IPW Stabalized  0.311352   
5  Propensity Score Matching -0.179289   
6          Distance Matching  0.630322   
7                     IPW LR  0.149171   

                                                 ITE    Library  other  
0                                                NaN      DoWhy    NaN  
1  [[0.        ]\n [0.        ]\n [0.        ]\n ...      DoWhy    NaN  
2                                                NaN      DoWhy    NaN  
3                                                NaN      DoWhy    NaN  
4                                                NaN      DoWhy    NaN  
5                                                NaN      DoWhy    NaN  
6                                                NaN      DoWhy    NaN  
7                                    

In [7]:
%store -r df_synthetic_results

method = "IPW LR"

df_synthetic_results = df_synthetic_results._append({'method': method, 'ATE': effect.values[0]}, ignore_index=True)

print(df_synthetic_results)
%store df_synthetic_results

                    method  ATE                                       ITE
0              Causal Tree  2.0  [2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0]
1              cforest_mse  NaN                                       NaN
2             cforest_cmse  2.0  [2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0]
3       cforest_cmse_p=0.5  2.0  [2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0]
4  cforest_cmse_p=0.5_md=3  2.0  [2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0]
5            cforest_ttest  2.0  [2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0]
6                   IPW LR  2.0                                       NaN
Stored 'df_synthetic_results' (DataFrame)
